<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk%20/nlp/modelling_all_museum_groups_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyldavis

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
csv_files = [
    'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/rk_nationalmuseum.csv',
    'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/olonmus.csv',
    'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/echo_association.csv',
    'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/domderevnivoknavolok.csv',
    'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/museum_ptz.csv'
]

In [ ]:
df = pd.concat([pd.read_csv(url, usecols = ['text','date']) for url in csv_files], ignore_index=True)
df = df[df['text'].notna() & (df['text'].apply(lambda x: isinstance(x, str)))]

In [12]:
df2 = df2.rename(columns={'text': 'tokens'})
def get_year(x):
   x = x.split('-')[0]
   return  x
df2['year'] = df['date'].apply(get_year)
df2.to_csv('df_tokens.csv', index=False)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
data = []
tokens = pd.read_csv('df_tokens.csv')['tokens']
for indx, row in tokens.items():
  a = []
  try:
    r = row.split()
  except:
    pass
  for w in r:
    a.append(w)
  data.append(a)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_trigrams = make_trigrams(data)

dictionary = corpora.Dictionary(data_words_trigrams)
dictionary.filter_extremes(no_below=20, no_above=0.6)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
from gensim.models import LdaModel

dictionary = corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]
# Обучение модели LDA
num_topics = 10

lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Вывод тем и связанных с ними слов
for idx, topic in lda_model.print_topics(-1):
    print(f"Тема {idx + 1}: {topic}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Тема 1: 0.065*"рубль" + 0.053*"билет" + 0.033*"стоимость" + 0.029*"занятие" + 0.028*"экскурсия" + 0.028*"музей" + 0.016*"человек" + 0.011*"карта" + 0.011*"руб" + 0.011*"мероприятие"
Тема 2: 0.012*"это" + 0.010*"который" + 0.009*"век" + 0.007*"олонецкий" + 0.006*"предмет" + 0.006*"карельский" + 0.005*"карелия" + 0.005*"вязание" + 0.005*"время" + 0.004*"олонецкийнациональныймузей"
Тема 3: 0.045*"год" + 0.019*"вокнаволок" + 0.016*"война" + 0.012*"деревня" + 0.012*"день" + 0.009*"победа" + 0.008*"школа" + 0.008*"олонецкий" + 0.008*"великий" + 0.007*"олонец"
Тема 4: 0.039*"карелия" + 0.028*"музей" + 0.018*"год" + 0.015*"национальный" + 0.015*"карельский" + 0.014*"проект" + 0.013*"республика" + 0.010*"культура" + 0.008*"язык" + 0.008*"история"
Тема 5: 0.024*"музей" + 0.022*"программа" + 0.017*"игрушка" + 0.015*"ребёнок" + 0.014*"наш" + 0.012*"ждать" + 0.011*"день" + 0.011*"игра" + 0.009*"приглашать" + 0.009*"интерактивный"
Тема 6: 0.032*"конкурс" + 0.018*"кижский" + 0.016*"участие" + 0.013*"

In [25]:
from gensim.models import CoherenceModel

cm = CoherenceModel(model=lda_model, corpus=corpus, texts=data, coherence='c_v')

coherence_lda = cm.get_coherence()
print(coherence_lda)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.495553276933257


In [26]:
pyLDAvis.enable_notebook(local=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

pyLDAvis.display(vis)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
topics = [[(term, round(wt, 3))
               for term, wt in lda_model.show_topic(n)]
                   for n in range(0, lda_model.num_topics)]
topics_df = pd.DataFrame([[term for term, wt in topic]
                              for topic in topics],
                         index=['Тема '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Тема 1,Тема 2,Тема 3,Тема 4,Тема 5,Тема 6,Тема 7,Тема 8,Тема 9,Тема 10
0,рубль,это,год,карелия,музей,конкурс,день,выставка,музей,кукла
1,билет,который,вокнаволок,музей,программа,кижский,весь,кижи,наш,калитка
2,стоимость,век,война,год,игрушка,участие,поздравлять,выставочный,выставка,сказка
3,занятие,олонецкий,деревня,национальный,ребёнок,участник,ЭХО,зал,это,клуб
4,экскурсия,предмет,день,карельский,наш,олонец,пусть,пл,история,птичка
5,музей,карельский,победа,проект,ждать,фестиваль,рождение,музей,карелия,лыжа
6,человек,карелия,школа,республика,день,деревня,желать,работа,свой,каша
7,карта,вязание,олонецкий,культура,игра,победитель,счастие,мастер,который,азбука
8,руб,время,великий,язык,приглашать,клуб,жизнь,программа,первый,цыплёнок
9,мероприятие,олонецкийнациональныймузей,олонец,история,интерактивный,команда,этнокультурный,телефон,день,угол
